# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
#приведем возраст и зарплату к int типу данных
df['Возраст'] = df['Возраст'].astype(int)
df['Зарплата'] = df['Зарплата'].astype(int)
df.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

In [6]:
#Посмотрим на количество дубликатов в таблице
df.duplicated().sum()

153

In [17]:
df.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


В представленном датафрейме 5 столбцов и 5000 строк. Дубликаты не удалила, т.к. целевой признак (страховые выплаты) не уникален и возможны естественные дубликаты.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:**  Качество линейной регрессии **не изменится** при умножении признаков на обратимую матрицу

**Обоснование:** 
Пусть качество линейной регрессии не меняется при умножении признаков (X) на обратимую матрицу (P), таком случае, подставим в формулы XP вместо X

1. Запишем предсказания в зашифрованной матрице [1]:  
$ a_1 = XP w $   

2. Выпишем уравнение для получения параметров линейной регрессии [2] исходя из $w = (X^T X)^{-1}X^Ty$:  
$ w = ((XP )^T XP )^{-1} (X P )^T y$

3. Подставим уравнение [2] в [1]:  
$ a_1 = XP ((XP )^T XP )^{-1} (XP )^T y $

4. Применим свойство: $ (AB)^T = B^TA^T $:  
$ a_1 = XP ( P ^T X^T XP )^{-1} P ^T X^T y $

5. Учитывая, что $X^T X$ - квадратная матрица, и $ (ABC)^{-1} = C^{-1}B^{-1}A^{-1} $   
$ a_1 = XP  P ^{-1}(X^T X)^{-1}( P ^T)^{-1} P ^T X^T y $

6. Сокращение: исходим из свойства $AE = A$:    
$ a_1 = X E (X^T X)^{-1} E X^T y $   
 


7. Получаем исходную формулу:  
$ a_1 = X (X^T X)^{-1} X^T y  = Xw $

Следовательно:

$ Xw = XPw $

т.е. если умножить признаки на обратимую матрицу, качество линейной регрессии не изменится

## Алгоритм преобразования

**Алгоритм**

Чтобы защитить данные и не потерять качество модели машинного обучения, нужно умножить признаки на обратимую матрицу P. Для этого алгоритм преобразования данных должен содержать процесс создания подобной матрицы P и ее умножения на матрицу признаков.

**Обоснование**


In [7]:
# фиксируем генератор случайных чисел для воспроизводимости резульататов
np.random.seed(19191)

# матрица признаков
X = np.array([
    [1, 11, 12, 13],
    [1, 13, 22, 23],
    [1, 15, 32, 33],
    [1, 17, 42, 43],
    [1, 19, 52, 73],
    [1,21, 62, 83],
])


n = X.shape[1]

# генерируем ключ для шифрования
K_enc = np.random.normal(loc=0., scale=0.25, size=(n,n))

# генерируем ключ для дешифрования
K_dec = np.linalg.inv(K_enc)

In [8]:
X_enc = X @ K_enc

X_enc

array([[  0.79352324,  -6.92589821,  -0.93489649,  -9.04451785],
       [ -0.72123868, -10.20402163,  -0.93170813, -17.730097  ],
       [ -2.2360006 , -13.48214505,  -0.92851978, -26.41567614],
       [ -3.75076251, -16.76026847,  -0.92533142, -35.10125529],
       [-11.17790599, -18.38696056,  -0.10118061, -49.29444641],
       [-12.69266791, -21.66508398,  -0.09799225, -57.98002555]])

In [9]:
#Дешифровка данных
X_dec = X_enc @ K_dec
X_dec = np.round(X_dec).astype(int)
X_dec

array([[ 1, 11, 12, 13],
       [ 1, 13, 22, 23],
       [ 1, 15, 32, 33],
       [ 1, 17, 42, 43],
       [ 1, 19, 52, 73],
       [ 1, 21, 62, 83]])

Таким образом, после дешифровки, мы получили исходную матрицу, следовательно, алгоритм работает

## Проверка алгоритма

In [10]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=19191)

In [12]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_origin_data = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_origin_data)

R2 = 0.420128836005348


Преобразуем матрицу признаков

In [13]:
n = features.shape[1]
np.random.seed(19191)
enc_matrix = np.random.normal(loc=0., scale=0.25, size=(n,n))
dec_matrix = np.linalg.inv(K_enc)
enc_features = features @ enc_matrix
enc_matrix

array([[ 0.57734914,  0.05230066, -0.479714  ,  0.17963807],
       [ 0.27087296, -0.36360725, -0.05814612,  0.17138601],
       [ 0.08996829, -0.33766246, -0.02910006, -0.62745452],
       [-0.29561908,  0.08257157,  0.04104812, -0.2753806 ]])

In [14]:
enc_features_train, enc_features_test = train_test_split(enc_features, test_size=0.25, random_state=19191)

In [15]:
model = LinearRegression()
model.fit(enc_features_train, target_train)
R2_encoded_data = r2_score(target_test, model.predict(enc_features_test))
print("R2 =", R2_encoded_data)

R2 = 0.42012883600542084


На примере показателя R2 мы видим, что качество линейной регрессии не отличается до и после преобразования

## Вывод:

Для защиты данных клиентов страховой компании «Хоть потоп» мы разработали метод преобразования данных.
1. В качестве исходных данных были использованы данные клиентов с целевым признаком: количество страховых выплат клиенту за последние 5 лет. Данные были приведены к типу int. Мультиколлинеарность в данных не выявлена.
2. Было доказано: при умножении признаков на обратимую матрицу, качество линейной регрессии не изменится.
2. Для преобразования данных был написан алгоритм, содержащий процесс создания обратимой матрицы (np.random.normal) и ее умножения на матрицу признаков. 
3. При этом, с преобразованием данных, качество моделей машинного обучения не ухудшилось: для линейной регрессии показатель R2 остался прежним: 0.4201288.
